In [ ]:
import copy
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
from tqdm.auto import tqdm
import xarray as xr

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
n_ens = 1001

In [ ]:
pd.options.display.max_columns = 50
df_configs = pd.read_csv('../data/fair2.1-parameters/calibrated_constrained_parameters_v1.1.0.csv', index_col=0)
valid_all = list(df_configs.index)[:n_ens]
df_configs

In [ ]:
df_solar = pd.read_csv(
    "../data/forcing/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/forcing/volcanic_ERF_monthly_-950001-201912.csv"
)

In [ ]:
volcanic_forcing = np.zeros(551)
for i, year in enumerate(np.arange(1750, 2021)):
    volcanic_forcing[i] = np.mean(
        df_volcanic.loc[
            ((year - 1) <= df_volcanic["year"]) & (df_volcanic["year"] < year)
        ].erf
    )
volcanic_forcing[271:281] = np.linspace(1, 0, 10) * volcanic_forcing[270]

In [ ]:
solar_forcing=np.zeros(551)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf_timebounds.csv', index_col=0)
solar_forcing[:281] = solar_forcing_df['erf'].values[:281]
solar_forcing[271:281] = np.linspace(1, 0, 10) * solar_forcing[271:281]

In [ ]:
trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/CH4_lifetime.csv",
    index_col=0,
)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2300, 1)
scenarios = ['SSP1-19', 'SSP5-34', 'EN_NPi2020_400f_lowBECCS']
f.define_scenarios(scenarios)
species, properties = read_properties()
f.define_species(species, properties)
f.define_configs(list(valid_all))
f.allocate()

In [ ]:
coffee = pd.read_csv('../data/emissions/COFFEE1.1___EN_NPi2020_400f_lowBECCS.csv', index_col=0)
ssp119 = pd.read_csv('../data/emissions/ssp119.csv', index_col=0)
ssp534 = pd.read_csv('../data/emissions/ssp534.csv', index_col=0)

In [ ]:
# add in emissions
for specie in ssp119.index:
    f.emissions.loc[dict(scenario='SSP1-19', specie=specie)] = ssp119.loc[specie, :].interpolate().values[:-1][:, None]
for specie in ssp534.index:
    f.emissions.loc[dict(scenario='SSP5-34', specie=specie)] = ssp534.loc[specie, :].interpolate().values[:-1][:, None]
for specie in coffee.index:
    f.emissions.loc[dict(scenario='EN_NPi2020_400f_lowBECCS', specie=specie)] = coffee.loc[specie, :].values[:-1][:, None]

In [ ]:
# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs.loc[valid_all, "scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs.loc[valid_all, "solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs.loc[valid_all, "solar_trend"].values.squeeze(),
    specie="Solar",
)

# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[valid_all, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[valid_all, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs.loc[valid_all,"epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs.loc[valid_all,"gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs.loc[valid_all,"sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs.loc[valid_all,"sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs.loc[valid_all,"seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[valid_all,"F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[valid_all,"r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"], df_configs.loc[valid_all,"rA"].values.squeeze(), specie="CO2"
)
fill(f.species_configs["iirf_uptake"], df_configs.loc[valid_all,"rU"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[valid_all,"rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[valid_all,"beta"].values.squeeze())
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[valid_all,"shape Sulfur"].values.squeeze(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"], df_configs.loc[valid_all,"shape BC"].values.squeeze(), specie="BC"
)
fill(
    f.species_configs["aci_shape"], df_configs.loc[valid_all,"shape OC"].values.squeeze(), specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[valid_all,f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,f"scale {specie}"].values.squeeze(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[valid_all,"scale minorGHG"].values.squeeze(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[valid_all,f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[valid_all,"co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

f.run()

In [ ]:
temp_norm_19952014 = np.ones(21)
temp_norm_19952014[0] = 0.5
temp_norm_19952014[-1] = 0.5

temp_norm_18501900 = np.ones(52)
temp_norm_18501900[0] = 0.5
temp_norm_18501900[-1] = 0.5

In [ ]:
concentration_co2 = np.percentile(f.concentration[:, :, :, 2], (5, 50, 95), axis=2).transpose(1,2,0)
concentration_ch4 = np.percentile(f.concentration[:, :, :, 3], (5, 50, 95), axis=2).transpose(1,2,0)
forcing_ch4 = np.percentile(f.forcing[:, :, :, 3], (5, 50, 95), axis=2).transpose(1,2,0)
forcing_o3 = np.percentile(f.forcing[:, :, :, 58], (5, 50, 95), axis=2)   .transpose(1,2,0)
forcing_sum = np.percentile(f.forcing_sum, (5, 50, 95), axis=2).transpose(1,2,0)
temperature = np.percentile(f.temperature[:, :, :, 0] - np.average(
    f.temperature[100:152, :, :, 0],
    weights = temp_norm_18501900,
    axis=0
), (5, 50, 95), axis=2).transpose(1,2,0)

In [ ]:
concentration_co2.shape

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        concentration_co2 = (["timebound", "scenario", "quantile"], concentration_co2),
        concentration_ch4 = (["timebound", "scenario", "quantile"], concentration_ch4),
        forcing_ch4 = (["timebound", "scenario", "quantile"], forcing_ch4),
        forcing_o3 = (["timebound", "scenario", "quantile"], forcing_o3),
        forcing_sum = (["timebound", "scenario", "quantile"], forcing_sum),
        temperature = (["timebound", "scenario", "quantile"], temperature),
    ),
    coords = dict(
        timepoint=np.arange(1750.5, 2300),
        timebound=np.arange(1750, 2301),
        scenario=scenarios,
        quantile=[5, 50, 95]
    )
)

In [ ]:
ds

In [ ]:
os.makedirs('../results', exist_ok=True)

In [ ]:
ds.to_netcdf('../results/noCH4R_headlines.nc')